### Batches, Batch Normalization and Dropout

In this workbook you can experiment what you learnt about how to make batches out of your data, how to perform batch normalization and dropout

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import math

from torchvision import datasets, transforms
torch.manual_seed(44)

Load the data from data/batches_norm_drop.csv, then take a look at them.

In [31]:
# YOUR CODE HERE
data = pd.read_csv("/Users/szokirov/Documents/GitHub/strive_practice/Chapter 03/04. DataLoader/data/batches_norm_drop.csv", header=None)
print(data.head())
print(data[2].value_counts())
print(data.shape)

          0         1  2
0  0.350140  4.248592  0
1  0.950728  3.528855  0
2  1.371517  3.149416  0
3  0.268221  4.337209  0
4  1.881996  1.515387  0
0    250
1    250
2    250
Name: 2, dtype: int64
(750, 3)


Now you are ready to code your own function to create batches. If needed rewatch the video we provided in Eduflow.

**Extra challange:**    Are you able to split between train and test _**without**_ using sklearn?

In [44]:
def batchify(x_train, x_test, y_train, y_test, batch_size):
#YOUR CODE HERE
    num_batch = x_train.shape[0] // batch_size
    num_batch_test = x_test.shape[0] // batch_size

    indexes_train = np.random.permutation(x_train.shape[0])
    indexes_test = np.random.permutation(x_test.shape[0])

    x_train = x_train[indexes_train]
    x_test = x_test[indexes_test]

    y_train = y_train[indexes_train]
    y_test = y_test[indexes_test]

    x_train = x_train[ :batch_size * num_batch ].reshape(num_batch, batch_size, x_train.shape[1])
    y_train = y_train[ :batch_size * num_batch ].reshape(num_batch, batch_size, 1)

    x_test = x_test[ :batch_size * num_batch_test ].reshape(num_batch_test, batch_size, x_test.shape[1])
    y_test = y_test[ :batch_size * num_batch_test ].reshape(num_batch_test, batch_size, 1)

    return x_train, x_test, y_train, y_test


In [48]:
data = data.sample(frac=1)
x = data.iloc[:, :2].values
y = data.iloc[:, -1].values
#splitting the data
train_ratio = 0.8
n_train = math.floor(train_ratio * x.shape[0])
n_test = math.ceil((1-train_ratio) * x.shape[0])
x_train = x[:n_train]
y_train = y[:n_train]
x_test = x[n_train:]
y_test = y[n_train:]
print("Total Number of samples in train:",x_train.shape[0])
print("Total Number of samples in test:",x_test.shape[0])

Total Number of samples in train: 600
Total Number of samples in test: 150


In [45]:
#batch
x_train_batch, x_test_batch, y_train_batch, y_test_batch = batchify(x_train, x_test, y_train, y_test, batch_size=16)

It's time to create your model! Remember to include the new tricks you learnt (batch normalization and dropout)!

In [55]:
# YOUR CODE HERE
class Network(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], 1)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        # x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [56]:
hidden_size = [20, 10]
model = Network(2, hidden_size)
model

Network(
  (fc1): Linear(in_features=2, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

Train your model and evaluate it. **Extra challenge**: try to figure out how you can tell if batch norm and dropout are effective

In [57]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()
num_epoch = 5
for epoch in range(num_epoch):
    train_loss = []
    test_loss = []
    model.train()
    print(f"Starting epoch{epoch}")
    current_loss = 0
    for batch, (x_train_batch, y_train_batch) in enumerate(zip(x_train, y_train)):
        optimizer.zero_grad()
        train_preds = model(x_train_batch)
        losses = loss(train_preds, y_train_batch)
        current_loss += losses.item()
        loss.backward()
        optimizer.step()
        if batch % 1000==0:
            print('Train Epoch: {} \tLoss: {:.8f}'.format(epoch,loss.item()))
        train_loss.append(current_loss/x_train.shape[0])
    model.eval()
    with torch.no_grad():
        running_loss = 0
        for batch_id, (x_test_batch, y_test_batch) in enumerate(zip(x_test, y_test)):
            test_preds = model(x_test_batch)
            losses = loss(test_preds, y_test_batch)
            running_loss += losses.item()
        test_loss.append(running_loss/x_test.shape[0])
    model.eval()

    

Starting epoch0


TypeError: 'int' object is not callable